In [1]:
#dependencies
import cv2
from PIL import Image
from scipy import spatial
import numpy as np
import os
import pandas as pd
from rembg import remove
import pickle

In [2]:
images_filepath = '/Users/countrygirl411/Desktop/Image Dataset 2/Kyra'
cfd_filepath = '/Users/countrygirl411/Desktop/CFD Version 3.0/Images'

In [3]:
#Get tuples of images
def sd_filelist():
    sd_images = {}
    prompts = {}
    for file in os.listdir(images_filepath):
        if not file.startswith('.'):
            num = int(file.split("-")[0])
            p = ''.join(file.split("-")[1:]).split(".")[0]
            if num in sd_images:
                sd_images[num].append(file)
                if len(p)+1 > len(prompts[num]):
                    prompts[num] = p.replace("frontfacing", "front-facing")
            else:
                sd_images[num] = [file]
                prompts[num] = p.replace("frontfacing", "front-facing")
    return sd_images, prompts

In [ ]:
def cfd_filelist():
    cfd = {}
    count = 0
    for file in os.listdir(cfd_filepath):
        if not file.startswith('.'):
            trait = file.split("-")[1]
            expression = file.split("-")[-1].split(".")[0]
            if trait+expression not in cfd:
                cfd[trait+expression] = [file]
            else:
                cfd[trait+expression].append(file)
    return cfd

In [ ]:
#Takes in a single image as numpy array, resizes and crops it
def crop_and_resize(img, out_width=512, out_height=512):
    height, width, channels = img.shape

    r = np.ceil(out_height / height)
    dim = (int(width * r), out_height)
    resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    height_resize, width_resize, channels = resized.shape
    
    crop_min = int((width_resize-out_width)*0.5)
    crop_max = int(out_width+(width_resize-out_width)*0.5)
    cropped = resized[:, crop_min:crop_max]

    #img_array2 = cropped.flatten()
    #img_array2 = cropped/255
    return cropped

In [ ]:
#CFD images
cfd = {}
count = 0
for file in os.listdir(cfd_filepath):
    if not file.startswith('.'):
        count += 1
        trait = file.split("-")[1]
        expression = file.split("-")[-1].split(".")[0]

        img = cv2.imread(f'{cfd_filepath}/{file}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width, channels = img.shape

        r = 512 / height
        dim = (int(width * r), 512)
        resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
        height_resize, width_resize, channels = resized.shape
        
        crop_min = int((width_resize-512)*0.5)
        crop_max = int(512+(width_resize-512)*0.5)
        cropped = resized[:, crop_min:crop_max]

        #img_array2 = cropped.flatten()
        img_array2 = cropped/255

        #Checking
        #if count in (range(5)):
            #i = Image.fromarray(img)
            #r = Image.fromarray(resized)
            #c = Image.fromarray(cropped)
            #i.show()
            #r.show()
            #c.show()
            #print(img_array2)
            #print(img_array2)
        
        if trait+expression not in cfd:
            cfd[trait+expression] = [img_array2]
        else:
            cfd[trait+expression].append(img_array2)

In [ ]:
#Check
for key in cfd.keys():
    print(f'{key}: {len(cfd[key])}')

In [4]:
# Image slicing
def cut_images(image_names):
    images = []
    for image_name in image_names:
        img = cv2.imread(f'{images_filepath}/{image_name}')
        #img = cv2.imread(image_name)
        img2 = img
    
        height, width, channels = img.shape
    
        # Number of pieces Horizontally
        W_SIZE  = 5
        # Number of pieces Vertically to each Horizontal
        H_SIZE = 5
    
        counter = 0

        ###NEED TO EXCLUDE ALL BLACK IMAGES
    
        for ih in range(H_SIZE ):
            for iw in range(W_SIZE ):
              x = width/W_SIZE * iw
              y = height/H_SIZE * ih
              h = (height / H_SIZE)
              w = (width / W_SIZE )
              #print(x,y,h,w)
              counter+=1
              img = img[int(y):int(y+h), int(x):int(x+w)]
              #cv2.imwrite(str(counter) +  ".png",img)
              if not (img.max==0 and img.min==0):
                  images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) 
              img = img2
    return images

In [ ]:
#Test
images = cut_images(sd_images[0])
print(len(images))
print(images[0:5])
#for i in range(20):
#    img = Image.fromarray(images[i])
#    img.show()

In [ ]:
#Check
images_list[0].show()

In [ ]:
#df = pd.DataFrame(columns=['prompt_num', 'prompt'].extend(list(cfd.keys())))

cos_sim_dict = {'prompt':[], 'prompt_num':[]}

for i in range(len(list(sd_images.keys()))):
    key = list(sd_images.keys())[i]
    images_list = cut_images(sd_images[key])
    cos_sim_dict['prompt_num'].append(key)
    cos_sim_dict['prompt'].append(prompts[key])
    print(prompts[key])
        
    for cat in cfd.keys():
        print(len(cfd[cat]),len(images_list))
        cat_score = 0
        if cat not in cos_sim_dict:
            cos_sim_dict[cat] = []

        for img in images_list:
            img_array1 = img.flatten()
            img_array1 = img_array1/255
            
            for cfd_img in cfd[cat]:
                print(img_array1.shape, cfd_img.shape)
                similarity = -1 * (spatial.distance.cosine(img_array1, cfd_img) - 1)
                cat_score += similarity
                
        cat_avg = cat_score/(len(cfd[cat])*len(images_list))
        print(cat, cat_avg)
        cos_sim_dict[cat].append(cat_avg)

    if i%5==0 or i==len(list(sd_images.keys()))-1:
        print(cos_sim_dict)
        df = pd.DataFrame.from_dict(cos_sim_dict)
        df.to_csv(f'CFD_cos_similarity_{i}.tsv', sep='\t', index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#Plot distributions of scores
cos_sim = pd.read_csv('CFD_cos_similarity_20.tsv', sep='\t')
#Have to reshape for better plotting
cos_sim = pd.melt(cos_sim, id_vars=['prompt', 'prompt_num'])
cos_sim['gender'] = cos_sim['variable'].astype(str).str[1]
cos_sim['race'] = cos_sim['variable'].astype(str).str[0]
cos_sim['expression'] = cos_sim['variable'].astype(str).str[2:]

gplt = sns.stripplot(data=cos_sim, y='prompt', x='value', hue='gender')
plt.show()
rplt = sns.stripplot(data=cos_sim, y='prompt', x='value', hue='race')
plt.show()
eplt = sns.stripplot(data=cos_sim, y='prompt', x='value', hue='expression')
plt.show()

a4_dims = (32, 8)
fig, ax = plt.subplots(figsize=a4_dims)
allplt = sns.stripplot(data=cos_sim, y='prompt', x='value', ax=ax)
for line in range(0,cos_sim.shape[0]):
     allplt.text(cos_sim.value[line]+0.0001, cos_sim.prompt[line], 
     cos_sim.variable[line], horizontalalignment='left', 
     fontsize=8, color='black')
plt.show()

In [ ]:
#PCA
dim = (64, 64)

all_images = []
for key in sd_images.keys():
    images_list = cut_images(sd_images[key])
    images_list = [images_list[i] for i in range(len(images_list)) if i%10==0]
    images_list = [cv2.resize(img, dim, interpolation=cv2.INTER_AREA) for img in images_list]
    images_list = [img.flatten()/255 for img in images_list]
    all_images.extend(images_list)

for cat in cfd.keys():
    for i in range(len(cfd[cat])):
        if i%5==0:
            #print(cfd[cat][i].shape)
            pic = cv2.resize(cfd[cat][i], dim, interpolation=cv2.INTER_AREA).flatten()
            all_images.append(pic)
print(len(all_images))

In [ ]:
print([len(img) for img in all_images])
#print(all_images[-1])

In [ ]:
from sklearn import cluster, decomposition
import seaborn as sns
import matplotlib.pyplot as plt

comps = []
#print(all_images[0:5])
all_images = np.array(all_images)
for i in range(10, 101, 10):
    print(i)
    pca_estimator = decomposition.PCA(n_components=i, svd_solver="randomized", whiten=True)
    pca_estimator.fit(all_images)
    comps.append(sum(pca_estimator.explained_variance_ratio_))

sns.lineplot(x=np.array(list(range(10, 101, 10))), y=np.array(comps))
plt.xlabel('PCA components')
plt.ylabel('Explained Variance Ratio')

In [ ]:
pca_estimator = decomposition.PCA(n_components=100, svd_solver="randomized", whiten=True)
pca_estimator.fit(all_images)



In [ ]:
##PCA dimensions
dim=(64,64)
df = pd.DataFrame(columns=['prompt_num', 'prompt'].extend(list(cfd.keys())))

cos_sim_dict = {'prompt':[], 'prompt_num':[]}

for i in range(len(list(sd_images.keys()))):
    key = list(sd_images.keys())[i]
    images_list = cut_images(sd_images[key])
    cos_sim_dict['prompt_num'].append(key)
    cos_sim_dict['prompt'].append(prompts[key])
    print(prompts[key])
        
    for cat in cfd.keys():
        print(len(cfd[cat]),len(images_list))
        cat_score = 0
        if cat not in cos_sim_dict:
            cos_sim_dict[cat] = []

        for img in images_list:
            img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
            img_array1 = img.flatten()
            img_array1 = img_array1/255
            img_array1 = pca_estimator.transform(img_array1.reshape(1, -1))
            
            for cfd_img in cfd[cat]:
                cfd_img = cv2.resize(cfd_img, dim, interpolation=cv2.INTER_AREA)
                cfd_img = cfd_img.flatten()
                cfd_img = pca_estimator.transform(cfd_img.reshape(1, -1))
                #print(img_array1, cfd_img)
                similarity = -1 * (spatial.distance.cosine(img_array1[0], cfd_img[0]) - 1)
                cat_score += similarity
                
        cat_avg = cat_score/(len(cfd[cat])*len(images_list))
        print(cat, cat_avg)
        cos_sim_dict[cat].append(cat_avg)

    #print(cos_sim_dict)
    df = pd.DataFrame.from_dict(cos_sim_dict)
    df.to_csv(f'CFD_PCA_cos_similarity.tsv', sep='\t', index=False)

In [ ]:
def image_grid(imgs, rows, cols):
    #assert len(imgs) == rows*cols

    #w, h = imgs[0].size
    w,h = (512, 512)
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
#Detect and crop face from single image as numpy
desktop = "/Users/countrygirl411/Desktop/"
cascades = [f'{desktop}haarcascade_frontalface_default.xml', f'{desktop}haarcascade_frontalface_alt.xml', f'{desktop}haarcascade_frontalface_alt2.xml', f'{desktop}haarcascade_frontalface_alt_tree.xml', f'{desktop}haarcascade_profileface.xml']
height = 512
width = 512

def face_detect(img):
    #Detect face using open CV
    img_cv = img
    #Normal image
    img_pil = Image.fromarray(img_cv)

    #Image with removed background
    output = remove(img_pil)
    output = np.array(output)
    # Convert RGB to BGR
    #output = output[:, :, ::-1].copy()

    gray_image = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    face = []
    detected = []
    removed = []

    #output = remove(img_pil)

    for c in cascades:
        if len(face)==0:
            face_classifier = cv2.CascadeClassifier(c)
            #May have to adjust these parameters to avoid false positives
            face = face_classifier.detectMultiScale(
                gray_image, scaleFactor=1.1, minNeighbors=10, minSize=(int(height*0.05), int(width*0.05))
            )
        else:
            break

    if len(face) == 0:
        detected = img_pil
        removed = img_pil
        img_pil = img_pil

    #Crop to get just face
    else:
        #for (x, y, w, h) in face: 
        (x, y, w, h) = face[0]
        faces = output[y:y + h, x:x + w]
        img_pil = Image.fromarray(faces) #small face with crop and removed
        #Remove background
        #output = remove(img_pil)
        removed = Image.fromarray(output) #original img with background removed
        detected = Image.fromarray(cv2.rectangle(img_cv, (x, y), (x+w, y+h), (0, 0, 255), 2)) #original img with box around face
    return face, detected, img_pil, removed

In [ ]:
#Import image
#file = ['5-a front-facing photo of a person who is blind completely.png']
#img = cv2.imread(f'{file}')
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#height, width, channels = img.shape

#Calculate amount of white/distribution of color pixels
#Compute cosine similarities

#Other things that you might want to do:
#When to downsample?
#How many faces should we take from each image?
#Check output of face detection
#Save cropped/processed images separately
#Check for black images?

desktop = "/Users/countrygirl411/Desktop/"
cascades = [f'{desktop}haarcascade_frontalface_default.xml', f'{desktop}haarcascade_frontalface_alt.xml', f'{desktop}haarcascade_frontalface_alt2.xml', f'{desktop}haarcascade_frontalface_alt_tree.xml', f'{desktop}haarcascade_profileface.xml']

faces = {}

for j in range(len(list(sd_images.keys()))):
    key = list(sd_images.keys())[j]
    print(key)
    images = cut_images(sd_images[key])
    
    cropped_images = []
    removed_images = []
    detected_images = []

    faces[key] = []
    
    for i in range(len(images)):
        face, cropped, detected_2, removed = face_detect(images[i])
        cropped_images.append(cropped)
        removed_images.append(removed)
        detected_images.append(detected_2)
        faces[key].append(face)
    
    grid1 = image_grid(cropped_images, rows=5, cols=10)
    grid1.save(f'{key}_faces_box.png')
    grid2 = image_grid(removed_images, rows=5, cols=10)
    grid2.save(f'{key}_removed.png')
    grid3 = image_grid(detected_images, rows=5, cols=10)
    grid3.save(f'{key}_faces.png')
    
    # Open a file and use dump() 
    with open('sd_faces.pkl', 'wb') as file: 
        # A new file will be created 
        pickle.dump(faces, file)

In [ ]:
desktop = "/Users/countrygirl411/Desktop/"
cascades = [f'{desktop}haarcascade_frontalface_default.xml', f'{desktop}haarcascade_frontalface_alt.xml', f'{desktop}haarcascade_frontalface_alt2.xml', f'{desktop}haarcascade_frontalface_alt_tree.xml', f'{desktop}haarcascade_profileface.xml']

faces = {}

cfd = cfd_filelist()
for j in range(len(list(cfd.keys()))):
    key = list(cfd.keys())[j]
    print(key)

    detected_images = []
    removed_images = []
    detected_2_images = []

    for file in cfd[key]:
        img = cv2.imread(f'{cfd_filepath}/{file}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = crop_and_resize(img, out_width=512, out_height=512)
        face, detected, detected_2, removed = face_detect(img)

        faces[file] = face
        detected_images.append(detected)
        removed_images.append(removed)
        detected_2_images.append(detected_2)

    rows = int(np.floor(np.sqrt(len(detected_images))))
    cols = int(len(detected_images)/rows + 1)
    grid1 = image_grid(detected_images, rows=rows, cols=cols)
    grid1.save(f'cfd_{key}_faces_box.png')
    grid2 = image_grid(removed_images, rows=rows, cols=cols)
    grid2.save(f'cfd_{key}_removed.png')
    grid3 = image_grid(detected_2_images, rows=rows, cols=cols)
    grid3.save(f'{key}_faces.png')

    # Open a file and use dump() 
    with open('cfd_faces.pkl', 'wb') as file: 
        # A new file will be created 
        pickle.dump(faces, file)

In [ ]:
#Calculate cosine similarities of detected faces for CFD
# open a file, where you stored the pickled data
cfd_file = open('/Users/countrygirl411/Desktop/cfd_faces/cfd_faces.pkl', 'rb')
sd_file = open('/Users/countrygirl411/Desktop/sd_faces/sd_faces.pkl', 'rb')

# dump information to that file
cfd_data = pickle.load(cfd_file)
sd_data = pickle.load(sd_file)

# close the file
cfd_file.close()
sd_file.close()
#In call to cosine similarity... pass in images? crops? 

In [ ]:
def cfd_faces():
    faces = []
    cats = []
    count = 1
    for cfd_key in cfd_data.keys():
        if count % 100 == 0:
            print(count)
        count += 1
        cat = ''.join([cfd_key.split('-')[1], cfd_key.split('-')[-1].split(".")[0]])
        #print(cfd_key, cat)
    
        #Load sd_images and cfd_image
        cfd_img = cv2.imread(f'{cfd_filepath}/{cfd_key}')
        cfd_img = cv2.cvtColor(cfd_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(cfd_img)
        #Image with removed background
        cfd_img = remove(img_pil)
        cfd_img = np.array(cfd_img)
        cfd_img = crop_and_resize(cfd_img, out_width=512, out_height=512)

        if len(cfd_data[cfd_key])>0:
            (x_c, y_c, w_c, h_c) = cfd_data[cfd_key][0]
            cfd_crop = cfd_img[y_c:y_c + h_c, x_c:x_c + w_c]
    
            trans_mask = cfd_crop[:,:,3] <= 200
            cfd_crop[trans_mask] = [127, 127, 127, 127]
            cfd_crop = cv2.cvtColor(cfd_crop, cv2.COLOR_BGRA2BGR)
            
            faces.append(cfd_crop)
            cats.append(cat)
    return faces, cats

cfd_faces_out, cfd_cats_out = cfd_faces()

with open('cfd_faces_crop.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump((cfd_faces_out, cfd_cats_out), file)
file.close()

In [ ]:
sd_images = sd_filelist()[0]
#Cos_sim_dict has xxx entries-prompt, prompt_num, and xxx cfd categories
#prompt and prompt_num are the length of sd_images
#xxx category -> dictionary where the prompt num contains list of similarity scores for an image? 
cos_sim_dict = {}
sd_faces_crop = {}

cfd_faces_file = open('/Users/countrygirl411/Desktop/cfd_faces_crop.pkl', 'rb')
# dump information to that file
cfd_faces_out, cfd_cats_ou = pickle.load(cfd_faces_file)
# close the file
cfd_faces_file.close()

keys = list(sd_data.keys())
keys.sort()

for sd_key in keys:
    print(int(sd_key))
    #cos_sim_dict['prompt_num'].append(sd_key)
    #cos_sim_dict['prompt'].append(prompts[sd_key])
    cos_sim_dict[sd_key] = {}
    sd_faces_crop[sd_key] = []
    sd_imgs = cut_images(sd_images[int(sd_key)])

    for i in range(len(sd_imgs)):
        sd_img = Image.fromarray(sd_imgs[i])
        #Image with removed background
        sd_img = remove(sd_img)
        sd_img = np.array(sd_img)

        # Threshold-> can be greater than zero. eg 0.006 //Grayscale
        gray = (np.abs(sd_img[0]-sd_img[1]) < 11).all() and (np.abs(sd_img[0]-sd_img[2])< 11).all() and (np.abs(sd_img[1]-sd_img[2])< 11).all()
        #Crop faces / remove background
        sd_faces = sd_data[sd_key][i]
        if len(sd_faces) > 0 and not gray:
            (x, y, w, h) = sd_faces[0]
            sd_crop = sd_img[y:y + h, x:x + w]
            trans_mask = sd_crop[:,:,3] <= 200
            sd_crop[trans_mask] = [127, 127, 127, 127]
            sd_crop = cv2.cvtColor(sd_crop, cv2.COLOR_BGRA2BGR)
            sd_faces_crop[sd_key].append(sd_crop)
            
            for j in range(len(cfd_faces_out)):
                cfd_crop = cfd_faces_out[j]
                cat = cfd_cats_out[j]
                #print(sd_crop.shape, cfd_crop.shape)

                if sd_crop.shape[0] > cfd_crop.shape[0]:
                    sd_crop = crop_and_resize(sd_crop, cfd_crop.shape[0], cfd_crop.shape[0])
                else:
                    cfd_crop = crop_and_resize(cfd_crop, sd_crop.shape[0], sd_crop.shape[0])
                #print(sd_crop.shape, cfd_crop.shape)

                #Compute RGB cosine similarity
                sd_flat = sd_crop.flatten()/255
                cfd_flat = cfd_crop.flatten()/255
                #print(sd_flat.shape, cfd_flat.shape)
                similarity = -1 * (spatial.distance.cosine(sd_flat, cfd_flat) - 1)

                if cat in cos_sim_dict[sd_key]:
                    cos_sim_dict[sd_key][cat].append(similarity)
                else:
                    cos_sim_dict[sd_key][cat] = [similarity]    

                #print(cos_sim_dict)
    
    with open('cos_sim_dict.pkl', 'wb') as file: 
        # A new file will be created 
        pickle.dump(cos_sim_dict, file)
        file.close()

    with open('sd_faces_crop.pkl', 'wb') as file: 
        # A new file will be created 
        pickle.dump(sd_faces_crop, file)
        file.close()

In [ ]:
sd_images = sd_filelist()[0]
#Cos_sim_dict has xxx entries-prompt, prompt_num, and xxx cfd categories
#prompt and prompt_num are the length of sd_images
#xxx category -> dictionary where the prompt num contains list of similarity scores for an image? 
cos_sim_dict = {}
sd_faces_crop = {}

cfd_faces_file = open('/Users/countrygirl411/Desktop/cfd_faces_crop.pkl', 'rb')
# dump information to that file
cfd_faces_out, cfd_cats_out = pickle.load(cfd_faces_file)
# close the file
cfd_faces_file.close()

sd_faces_file = open('/Users/countrygirl411/Desktop/sd_faces_crop.pkl', 'rb')
sd_faces_out = pickle.load(sd_faces_file)
# close the file
sd_faces_file.close()

keys = list(sd_faces_out.keys())
keys.sort()

for sd_key in keys:
    print(int(sd_key))
    cos_sim_dict[sd_key] = {}
    
    for i in range(len(sd_faces_out[sd_key])):
        sd_img = sd_faces_out[sd_key][i]
        gray = (np.abs(sd_img[0]-sd_img[1]) < 11).all() and (np.abs(sd_img[0]-sd_img[2])< 11).all() and (np.abs(sd_img[1]-sd_img[2])< 11).all()
        
        if not gray:
            for j in range(len(cfd_faces_out)):
                cfd_crop = cfd_faces_out[j]
                cat = cfd_cats_out[j]
                sd_crop = sd_img
                #print(sd_crop.shape, cfd_crop.shape)

                if sd_crop.shape[0] > cfd_crop.shape[0]:
                    sd_crop = crop_and_resize(sd_crop, cfd_crop.shape[0], cfd_crop.shape[0])
                else:
                    cfd_crop = crop_and_resize(cfd_crop, sd_crop.shape[0], sd_crop.shape[0])
                #print(sd_crop.shape, cfd_crop.shape)

                #Compute RGB cosine similarity
                sd_flat = sd_crop.flatten()/255
                cfd_flat = cfd_crop.flatten()/255
                #print(sd_flat.shape, cfd_flat.shape)
                similarity = -1 * (spatial.distance.cosine(sd_flat, cfd_flat) - 1)

                if cat in cos_sim_dict[sd_key]:
                    cos_sim_dict[sd_key][cat].append(similarity)
                else:
                    cos_sim_dict[sd_key][cat] = [similarity]   

    with open('cos_sim_dict.pkl', 'wb') as file: 
        # A new file will be created 
        pickle.dump(cos_sim_dict, file)
        file.close()

In [ ]:
#Need to count faces in sd_data -> that can all be done in a separate function

In [ ]:
from sklearn import cluster, decomposition
import seaborn as sns
import matplotlib.pyplot as plt

cos_sim_dict_avg = {}

for key in cos_sim_dict.keys():
    cos_sim_dict_avg[key] = {}
    for key_2 in cos_sim_dict[key]:
        cos_sim_dict_avg[key][key_2] = np.mean(cos_sim_dict[key][key_2])

cos_sim_df = pd.DataFrame.from_dict(cos_sim_dict_avg, orient='index')
cos_sim_df['prompt'] = prompts
cos_sim_df.to_csv(f'cos_similarity_cropped.tsv', sep='\t', index=False)

cos_sim = pd.melt(cos_sim_df, id_vars=['prompt'])
cos_sim['gender'] = cos_sim['variable'].astype(str).str[1]
cos_sim['race'] = cos_sim['variable'].astype(str).str[0]
cos_sim['expression'] = cos_sim['variable'].astype(str).str[2:]

cos_sim.to_csv(f'cos_similarity_cropped_melted.tsv', sep='\t', index=False)

a4_dims = (32, 16)
fig, ax = plt.subplots(figsize=a4_dims)
gplt = sns.stripplot(data=cos_sim[0:500], y='prompt', x='value', hue='gender')
plt.show()

a4_dims = (32, 16)
fig, ax = plt.subplots(figsize=a4_dims)
rplt = sns.stripplot(data=cos_sim[0:500], y='prompt', x='value', hue='race')
plt.show()

a4_dims = (32, 16)
fig, ax = plt.subplots(figsize=a4_dims)
eplt = sns.stripplot(data=cos_sim[0:500], y='prompt', x='value', hue='expression')
plt.show()

a4_dims = (32, 16)
fig, ax = plt.subplots(figsize=a4_dims)
allplt = sns.stripplot(data=cos_sim[0:500], y='prompt', x='value', ax=ax)
for line in range(0,cos_sim[0:500].shape[0]):
     allplt.text(cos_sim[0:500].value[line]+0.0001, cos_sim[0:500].prompt[line], 
     cos_sim[0:500].variable[line], horizontalalignment='left', 
     fontsize=8, color='black')
plt.show()

In [ ]:
#CLIP cosine similarity
import pickle
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

cfd_faces_file = open('/Users/countrygirl411/Desktop/cfd_faces_crop.pkl', 'rb')
cfd_faces_out, cfd_cats_out = pickle.load(cfd_faces_file)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

face_clip_features = []

for f in cfd_faces_out:
    img = Image.fromarray(f)
    inputs = processor(images=img, return_tensors="pt")
    image_features = model.get_image_features(**inputs)
    #print(f.shape, image_features.shape)
    face_clip_features.append(image_features.detach().numpy())

with open('cfd_clip_features.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump((cfd_cats_out, face_clip_features), file)
    file.close()

In [ ]:
sd_faces_file = open('/Users/countrygirl411/Desktop/sd_faces_crop.pkl', 'rb')
sd_faces_out = pickle.load(sd_faces_file)
sd_faces_file.close()

keys = list(sd_faces_out.keys())
keys.sort()

face_clip_features = {}

for sd_key in keys:
    print(sd_key)
    face_clip_features[sd_key] = []
    for f in sd_faces_out[sd_key]:
        img = Image.fromarray(f)
        inputs = processor(images=img, return_tensors="pt")
        image_features = model.get_image_features(**inputs)
        #print(f.shape, image_features.shape)
        face_clip_features[sd_key].append(image_features.detach().numpy())

with open('sd_clip_features.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump(face_clip_features, file)
    file.close()

In [ ]:
cfd_clip_file = open('/Users/countrygirl411/Desktop/cfd_clip_features.pkl', 'rb')
# dump information to that file
cfd_cats_out, cfd_clip_out = pickle.load(cfd_clip_file)
# close the file
cfd_clip_file.close()

sd_clip_file = open('/Users/countrygirl411/Desktop/sd_clip_features.pkl', 'rb')
sd_clip_out = pickle.load(sd_clip_file)
# close the file
sd_clip_file.close()

keys = list(sd_clip_out.keys())
keys.sort()
cos_sim_dict = {}

for sd_key in keys:
    print(int(sd_key))
    cos_sim_dict[sd_key] = {}

    for i in range(len(sd_clip_out[sd_key])):
        sd_img = sd_clip_out[sd_key][i]

        #turn into features so we don't know if img is gray or not at this pt 
        #gray = (np.abs(sd_img[0]-sd_img[1]) < 11).all() and (np.abs(sd_img[0]-sd_img[2])< 11).all() and (np.abs(sd_img[1]-sd_img[2])< 11).all()
        #print(gray)
        #if not gray:
        
        for j in range(len(cfd_clip_out)):
            cfd_crop = cfd_clip_out[j].squeeze()
            cat = cfd_cats_out[j]
            sd_crop = sd_img.squeeze()
            #print(cfd_crop.shape, sd_crop.shape)
            #print(sd_crop.shape, cfd_crop.shape)

            #if sd_crop.shape[0] > cfd_crop.shape[0]:
                #sd_crop = crop_and_resize(sd_crop, cfd_crop.shape[0], cfd_crop.shape[0])
            #else:
                #cfd_crop = crop_and_resize(cfd_crop, sd_crop.shape[0], sd_crop.shape[0])
            #print(sd_crop.shape, cfd_crop.shape)

            #Compute RGB cosine similarity
            #sd_flat = sd_crop.flatten()/255
            #cfd_flat = cfd_crop.flatten()/255
            #print(sd_flat.shape, cfd_flat.shape)
            similarity = -1 * (spatial.distance.cosine(sd_crop, cfd_crop) - 1)

            if cat in cos_sim_dict[sd_key]:
                cos_sim_dict[sd_key][cat].append(similarity)
            else:
                cos_sim_dict[sd_key][cat] = [similarity]

    with open('clip_cos_sim_dict.pkl', 'wb') as file: 
        # A new file will be created 
        pickle.dump(cos_sim_dict, file)
        file.close()

In [ ]:
from sklearn import cluster, decomposition
import seaborn as sns
import matplotlib.pyplot as plt

sd_images, prompts = sd_filelist()
cos_sim_dict_avg = {}

for key in cos_sim_dict.keys():
    cos_sim_dict_avg[key] = {}
    for key_2 in cos_sim_dict[key]:
        cos_sim_dict_avg[key][key_2] = np.mean(cos_sim_dict[key][key_2])

cos_sim_df = pd.DataFrame.from_dict(cos_sim_dict_avg, orient='index')
cos_sim_df['prompt'] = prompts
cos_sim_df.to_csv(f'clip_cos_similarity_cropped.tsv', sep='\t', index=False)

cos_sim = pd.melt(cos_sim_df, id_vars=['prompt'])
cos_sim['gender'] = cos_sim['variable'].astype(str).str[1]
cos_sim['race'] = cos_sim['variable'].astype(str).str[0]
cos_sim['expression'] = cos_sim['variable'].astype(str).str[2:]

cos_sim.to_csv(f'clip_cos_similarity_cropped_melted.tsv', sep='\t', index=False)

a4_dims = (25, 16)
fig, ax = plt.subplots(figsize=a4_dims)
gplt = sns.stripplot(data=cos_sim[0:500], y='prompt', x='value', hue='gender')
plt.show()

a4_dims = (25, 16)
fig, ax = plt.subplots(figsize=a4_dims)
rplt = sns.stripplot(data=cos_sim[0:500], y='prompt', x='value', hue='race')
plt.show()

a4_dims = (25, 16)
fig, ax = plt.subplots(figsize=a4_dims)
eplt = sns.stripplot(data=cos_sim[0:500], y='prompt', x='value', hue='expression')
plt.show()

a4_dims = (32, 16)
fig, ax = plt.subplots(figsize=a4_dims)
allplt = sns.stripplot(data=cos_sim[0:500], y='prompt', x='value', ax=ax)
for line in range(0,cos_sim[0:500].shape[0]):
     allplt.text(cos_sim[0:500].value[line]+0.0001, cos_sim[0:500].prompt[line], 
     cos_sim[0:500].variable[line], horizontalalignment='left', 
     fontsize=8, color='black')
plt.show()

In [ ]:
#Measuring prompt felicity: A) size of subject, B) depiction of other objects, 
#C) deviation of background from plain, D) coloration of the image, E) difference in gender/race ratios


In [ ]:
#For baseline images, cut images, remove background, crop faces, save cropped faces

crops = []
positions = []
baseline_path = '/Users/countrygirl411/Desktop/template images'
for file in os.listdir(baseline_path):
    print(file)
    imgs = cut_images([f'{baseline_path}/{file}'])
    for i in imgs:
        face, detected, img_pil, removed = face_detect(i)
        positions.append(face)
        crops.append(img_pil)

with open('baseline_crop.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump(crops, file)
    file.close()

with open('baseline_positions.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump(positions, file)
    file.close()

In [ ]:
#Baseline sizes

baseline_faces_file = open('baseline_positions.pkl', 'rb')
# dump information to that file
b_faces = pickle.load(baseline_faces_file)
# close the file
baseline_faces_file.close()

sizes = []
for f in b_faces:
    if len(f) > 0: #(x, y, w, h)
        s = f[0][2] * f[0][3]
        s = s / 512**2
        sizes.append(s)

print("Mean: ", np.mean(sizes))
print("STD: ", np.std(sizes))
print(len(sizes))

In [ ]:
#Prompt sizes
sd_faces_file = open('/Users/countrygirl411/Desktop/sd_faces/sd_faces.pkl', 'rb')
# dump information to that file
sd_faces = pickle.load(sd_faces_file)
# close the file
sd_faces_file.close()

keys = list(sd_faces.keys())
keys.sort()

face_size_dict = {'mean': [], 'std': [], 'prompts': [], 'prompt_num': list(range(93)), 'len': []}

for k in keys:
    sizes = []
    for f in sd_faces[k]:
        if len(f) > 0: #(x, y, w, h)
            s = f[0][2] * f[0][3]
            s = s / 512**2
            sizes.append(s)
    face_size_dict['mean'].append(np.mean(sizes))
    face_size_dict['std'].append(np.std(sizes))
    face_size_dict['len'].append(len(sizes))
    face_size_dict['prompts'].append(prompts[k])

df = pd.DataFrame(face_size_dict)
df.to_csv('sd_face_sizes.tsv', index=False, sep='\t')

In [ ]:
#Coloration-maybe only look at gray for now...

In [ ]:
#Baseline object detection
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

objects = []

baseline_path = '/Users/countrygirl411/Desktop/template images'
for file in os.listdir(baseline_path):
    print(file)
    imgs = cut_images([f'{baseline_path}/{file}'])
    for i in imgs:
        img_pil = Image.fromarray(i)
        results = model.predict(img_pil)
        objects.append(results)
        #print(results)

with open('baseline_objects.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump(objects, file)
    file.close()

In [ ]:
#SD object detection
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

imgs, prompts = sd_filelist()
keys = list(imgs.keys())
keys.sort()

objects = {}

for k in keys:
    print(k)
    objects[k] = []
    images = cut_images(imgs[k])
    for i in images:
        out = []
        img_pil = Image.fromarray(i)
        results = model.predict(img_pil, verbose=False)
        for res in results:
            out.append(res.boxes)
        objects[k].append(out)

with open('sd_objects.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump(objects, file)
    file.close()

In [13]:
#Count baseline objects

base_obj_file = open('baseline_objects.pkl', 'rb')
# dump information to that file
base_obj = pickle.load(base_obj_file)
# close the file
base_obj_file.close()

counts = {}

print(len(base_obj))
for b in base_obj:
    names = b[0].names
    boxes = b[0].boxes
    cls = boxes.cls.numpy()
    for c in cls:
        if names[int(c)] not in counts:
            counts[names[int(c)]] = 1
        else:
            counts[names[int(c)]] += 1

print(counts)

500
{'person': 535, 'car': 15, 'tie': 13, 'dog': 1, 'tennis racket': 1, 'cat': 1, 'donut': 2, 'traffic light': 2, 'tv': 1}


In [20]:
#Count SD objects
imgs, prompts = sd_filelist()

sd_obj_file = open('sd_objects.pkl', 'rb')
# dump information to that file
sd_obj = pickle.load(sd_obj_file)
# close the file
sd_obj_file.close()

counts = {}

for k in list(sd_obj.keys()):
    for b in sd_obj[k]:
        box = b[0]
        cls = b[0].cls.numpy()
        for c in cls:
            if names[int(c)] not in counts:
                counts[names[int(c)]] = [0 for i in range(93)]
            counts[names[int(c)]][k] += 1

counts['prompt_num'] = list(range(93))
counts['prompt'] = [prompts[p] for p in counts['prompt_num']]
df = pd.DataFrame(counts)
df['total'] = df.drop(columns=['prompt_num', 'person']).sum(axis=1, numeric_only=True)

df.to_csv('sd_obj_counts.tsv', index=False, sep='\t')

In [34]:
#Get individual race/gender breakdowns for each prompt
#For each prompt, go through each image and compare with cfd images
#For each prompt, save face no. and similarity with each of the categories
#Try not to kill yourself

cos_sim_dict_file = open('clip_cos_sim_dict.pkl', 'rb')
# dump information to that file
cos_sim_dict = pickle.load(cos_sim_dict_file)
# close the file
cos_sim_dict_file.close()

cfd_file = open('cfd_clip_features.pkl', 'rb')
# dump information to that file
cfd_faces = pickle.load(cfd_file)
# close the file
cfd_file.close()

sd_file = open('sd_clip_features.pkl', 'rb')
# dump information to that file
sd_faces = pickle.load(sd_file)
# close the file
sd_file.close()

#print(sd_faces)
#print(cfd_faces[1])


#How the fuck did these numbers even workkkk
#This is why I did averages
#I guess it's probably for each image, similarity with each face and then all in the same list
#So can figure out individual image breakdowns by looking at specific chunks of the list? 
print(len(cos_sim_dict[1]['BFN']), len(cos_sim_dict[1]['LFN']))
print(len(cos_sim_dict[2]['BFN']), len(cos_sim_dict[2]['LFN']))
print(len(sd_faces[1]), len(sd_faces[2]))
print(len(cos_sim_dict[1]['BFN'])/len(sd_faces[1]), len(cos_sim_dict[1]['LFN'])/len(sd_faces[1]))
print(len(cos_sim_dict[2]['BFN'])/len(sd_faces[2]), len(cos_sim_dict[2]['LFN'])/len(sd_faces[2]))

2080 1120
2184 1176
20 21
104.0 56.0
104.0 56.0


In [53]:
#Similarities loop through each image, and then each CFD images within that
#For each prompt
#Need how many images are of each race/gender--for each image, get category of CFD image that has the highest similarity
keys = list(cos_sim_dict.keys())
keys.sort()

max_dict = {}
for key in keys:
    cats = list(cos_sim_dict[key].keys())
    max_dict[key] = {}
    for cat in cats:
        prompt_len = len(cos_sim_dict[key][cat])/len(sd_faces[key])
        max_dict[key][cat] = []
        for i in range(len(cos_sim_dict[key][cat])):
            if i % prompt_len == 0:
                max_sim = 0
            sim = cos_sim_dict[key][cat][i]
            if sim > max_sim:
                max_sim = sim
            if (i+1) % prompt_len == 0:
                #print("True")
                max_dict[key][cat].append(max_sim)
#print(max_dict)

results = {}
for key in keys:
    results[key] = []
    for i in range(len(sd_faces[key])):
        max_sim = 0
        max_cat = ''
        for cat in max_dict[key].keys():
            sim = max_dict[key][cat][i]
            if sim > max_sim:
                max_sim = sim
                max_cat = cat
        results[key].append(max_cat)

#print(results)
out_dict = {}
for key in keys:
    r_list = [f'{r[0]}_race' for r in results[key]]
    race = {x:r_list.count(x) for x in r_list}
    g_list = [f'{r[1]}_gender' for r in results[key]]
    gender = {x:g_list.count(x) for x in g_list}
    e_list = [f'{r[2:]}_expression' for r in results[key]]
    expression = {x:e_list.count(x) for x in e_list}
    r = {**race, **gender, **expression}
    out_dict[key] = r
    
#print(out_dict)
out_df = pd.DataFrame.from_dict(out_dict, orient='index')
out_df['prompt_num'] = list(range(93))
out_df['prompt'] = [prompts[p] for p in counts['prompt_num']]
#print(out_df)
out_df.to_csv('cdc_comparison_counts.tsv', index=False, sep='\t')